## pytorch完整流程

### 数据读取
* 如果是一个类别一个文件夹，则可以使用ImageFolder
* 如果特殊数据集，则需要自定义一个数据集类，用于数据的加载、预处理等，并实现len，getitem等方法

In [1]:
import os
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import torch
import torch.optim as optim
from torch import nn
import torchvision
from torchvision import datasets, transforms, models
import imageio
import time
import warnings
import random
import sys
import copy
import json
from PIL import Image

### 先解压文件

In [9]:
import zipfile

zip_file_path = 'tiny-imagenet-200.zip'
extract_path = 'tiny-imagenet-200/'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

KeyboardInterrupt: 